0. Imports

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats # statistika

1. Load CSV

In [ ]:
# change to your file location
df = pd.read_csv('/content/drive/MyDrive/Škola/DM/evalvacia_modelu_IV_a/MLM_vstup.csv', ';', usecols=range(0,13))
df_stats = pd.read_csv('/content/drive/MyDrive/Škola/DM/evalvacia_modelu_IV_a/MLM_STUD_stats.csv', ';', usecols=range(0,10))

# fiter for students
df = df[(df['HODINA'] > 6) & (df['HODINA'] <= 22) & (df['STUD'] == 1) & (df['KATEGORIA'].isin(['uvod', 'studium', 'o_fakulte', 'oznamy']))]

# empty dict to save created crosstables
dfDict = {}

2. Create crosstables

*Crosstable - PO*

In [ ]:
df1 = df[(df['PO'] == 1)]
crosstable = pd.crosstab(df1['HODINA'], df1['KATEGORIA'], values=df1['PO'], margins=True, 
                 dropna=False, aggfunc='count').reset_index().fillna(0)

# Add missing line
crosstable = crosstable.append({'HODINA': 7, 'o_fakulte': 0, 'oznamy': 0, 'studium': 0, 'uvod': 0, 'All': 0}, ignore_index=True)

# Add PO crosstable into dict
dfDict['PO'] = crosstable
crosstable

KATEGORIA,HODINA,o_fakulte,oznamy,studium,uvod,All
0,8,3.0,0.0,3.0,13.0,19
1,9,3.0,15.0,8.0,61.0,87
2,10,5.0,12.0,16.0,59.0,92
3,11,0.0,9.0,14.0,69.0,92
4,12,19.0,11.0,5.0,74.0,109
5,13,8.0,4.0,6.0,67.0,85
6,14,2.0,5.0,27.0,90.0,124
7,15,3.0,7.0,110.0,53.0,173
8,16,1.0,1.0,13.0,9.0,24
9,17,0.0,3.0,7.0,6.0,16


*Crosstable - UT*

In [ ]:
df1 = df[(df['UT'] == 1)]
crosstable = pd.crosstab(df1['HODINA'], df1['KATEGORIA'], values=df1['UT'], margins=True, 
                 dropna=False, aggfunc='count').reset_index().fillna(0)

# Add UT crosstable into dict
dfDict['UT'] = crosstable

*Crosstable - STR*

In [ ]:
df1 = df[(df['STR'] == 1)]
crosstable = pd.crosstab(df1['HODINA'], df1['KATEGORIA'], values=df1['STR'], margins=True, 
                 dropna=False, aggfunc='count').reset_index().fillna(0)

# Add missing line   
crosstable = crosstable.append({'HODINA': 22, 'o_fakulte': 0, 'oznamy': 0, 'studium': 0, 'uvod': 0, 'All': 0}, ignore_index=True)

# Add STR crosstable into dict
dfDict['STR'] = crosstable

*Crosstable - STVR*

In [ ]:
df1 = df[(df['STVR'] == 1)]
crosstable = pd.crosstab(df1['HODINA'], df1['KATEGORIA'], values=df1['STVR'], margins=True, 
                 dropna=False, aggfunc='count').reset_index().fillna(0)

# Add STVR crosstable into dict
dfDict['STVR'] = crosstable

*Crosstable - PIA*

In [ ]:
df1 = df[(df['PIA'] == 1)]
crosstable = pd.crosstab(df1['HODINA'], df1['KATEGORIA'], values=df1['PIA'], margins=True, 
                 dropna=False, aggfunc='count').reset_index().fillna(0)

# Add missing lines
crosstable = crosstable.append({'HODINA': 18, 'o_fakulte': 0, 'oznamy': 0, 'studium': 0, 'uvod': 0, 'All': 0}, ignore_index=True)
crosstable = crosstable.append({'HODINA': 21, 'o_fakulte': 0, 'oznamy': 0, 'studium': 0, 'uvod': 0, 'All': 0}, ignore_index=True)

# Add PIA crosstable into dict
dfDict['PIA'] = crosstable

*Crosstable - SO*

In [ ]:
df1 = df[(df['SO'] == 1)]
crosstable = pd.crosstab(df1['HODINA'], df1['KATEGORIA'], values=df1['SO'], margins=True, 
                 dropna=False, aggfunc='count').reset_index().fillna(0)

# Add missing lines
crosstable = crosstable.append({'HODINA': 17, 'o_fakulte': 0, 'oznamy': 0, 'studium': 0, 'uvod': 0, 'All': 0}, ignore_index=True)
crosstable = crosstable.append({'HODINA': 18, 'o_fakulte': 0, 'oznamy': 0, 'studium': 0, 'uvod': 0, 'All': 0}, ignore_index=True)
crosstable = crosstable.append({'HODINA': 19, 'o_fakulte': 0, 'oznamy': 0, 'studium': 0, 'uvod': 0, 'All': 0}, ignore_index=True)
crosstable = crosstable.append({'HODINA': 20, 'o_fakulte': 0, 'oznamy': 0, 'studium': 0, 'uvod': 0, 'All': 0}, ignore_index=True)

# Add SO crosstable into dict
dfDict['SO'] = crosstable

3. Create collection of weekdays

In [ ]:
days = ['PO', 'UT', 'STR', 'STVR', 'PIA', 'SO']

4. Calculate differences

In [ ]:
# Dataframes for empirical relative abundance
df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
df4 = pd.DataFrame()

# Dataframes estimates for web parts
df1_estimate = pd.DataFrame()
df2_estimate = pd.DataFrame()
df3_estimate = pd.DataFrame()
df4_estimate = pd.DataFrame()
index = 0

# Cycle through hours from 7 to 23
for x in range (7,23):

      # Rows for empirical relative abundance
      new_row_uvod = {}
      new_row_studium = {}
      new_row_oznamy = {}
      new_row_fakulte = {}

      # Rows for estimations
      new_row_uvod_estimate = {}
      new_row_studium_estimate = {}
      new_row_oznamy_estimate = {}
      new_row_fakulte_estimate = {}
      i = 1

      # Cycle through weekdays
      for day in days:

        # Create logits estimates
        logit_uvod = df_stats.at[index, 'Intercept'] + df_stats.at[index, 'HODINA']*x+df_stats.at[index, 'HODINA_STV']*(x*x)+df_stats.at[index, day]
        logit_studium = df_stats.at[index+1, 'Intercept'] + df_stats.at[index+1, 'HODINA']*x+df_stats.at[index+1, 'HODINA_STV']*(x*x)+df_stats.at[index+1, day]
        logit_oznamy = df_stats.at[index+2, 'Intercept'] + df_stats.at[index+2, 'HODINA']*x+df_stats.at[index+2, 'HODINA_STV']*(x*x)+df_stats.at[index+2, day]
        
        reference_web = 1 / (np.exp(logit_uvod) + np.exp(logit_studium) + np.exp(logit_oznamy))
        
        # Create estimates for web parts
        estimate_uvod = np.exp(logit_uvod) * reference_web
        estimate_studium = np.exp(logit_studium) * reference_web
        estimate_oznamy = np.exp(logit_oznamy) * reference_web
        estimate_fakulte = np.exp(reference_web) * reference_web

        # Get current crosstable
        crosstable = dfDict[day]
        crosstable = crosstable[(crosstable['HODINA'] == x)]
        crosstable_all = crosstable.iloc[0]['All']

        # Empirical relative abundance
        if(crosstable_all == 0):
          dij_uvod = 0
          dij_studium = 0
          dij_oznamy = 0
          dij_fakulte = 0
        else:
          dij_uvod = crosstable.iloc[0]['uvod'] / crosstable_all
          dij_studium = crosstable.iloc[0]['studium'] / crosstable_all
          dij_oznamy = crosstable.iloc[0]['oznamy'] / crosstable_all
          dij_fakulte = crosstable.iloc[0]['o_fakulte'] / crosstable_all

        den = str(i) + '_' + day

        # Add data to new rows
        # Empirical
        new_row_uvod.update({den: dij_uvod})
        new_row_studium.update({den: dij_studium})
        new_row_oznamy.update({den: dij_oznamy})
        new_row_fakulte.update({den: dij_fakulte})

        # Estimations
        new_row_uvod_estimate.update({den: estimate_uvod})
        new_row_studium_estimate.update({den: estimate_studium})
        new_row_oznamy_estimate.update({den: estimate_oznamy})
        new_row_fakulte_estimate.update({den: estimate_fakulte})
        i = i + 1
      
       # Append time and ext to rows
      new_row_uvod.update({'0_hod': x})
      new_row_studium.update({'0_hod': x})
      new_row_oznamy.update({'0_hod': x})
      new_row_fakulte.update({'0_hod': x})
      new_row_uvod_estimate.update({'0_hod': x})
      new_row_studium_estimate.update({'0_hod': x})
      new_row_oznamy_estimate.update({'0_hod': x})
      new_row_fakulte_estimate.update({'0_hod': x})

      # Update dataframes
      df1 = df1.append(new_row_uvod, sort=False, ignore_index=True)
      df2 = df2.append(new_row_studium, sort=False, ignore_index=True)
      df3 = df3.append(new_row_oznamy, sort=False, ignore_index=True)
      df4 = df4.append(new_row_fakulte, sort=False, ignore_index=True)

      df1_estimate = df1_estimate.append(new_row_uvod_estimate, sort=False, ignore_index=True)
      df2_estimate = df2_estimate.append(new_row_studium_estimate, sort=False, ignore_index=True)
      df3_estimate = df3_estimate.append(new_row_oznamy_estimate, sort=False, ignore_index=True)
      df4_estimate = df4_estimate.append(new_row_fakulte_estimate, sort=False, ignore_index=True)

df1.head()

,0_hod,1_PO,2_UT,3_STR,4_STVR,5_PIA,6_SO
0,7.0,0.000000,0.600000,0.878788,1.000000,1.000000,0.636364
1,8.0,0.684211,0.333333,0.782609,0.750000,1.000000,0.888889
2,9.0,0.701149,0.946667,0.733333,0.645833,0.955056,0.933333
3,10.0,0.641304,0.633333,0.750000,0.478261,0.909091,1.000000
4,11.0,0.750000,0.537037,0.853333,0.210526,0.875000,0.972603


5. Create collection of weekdays with numbers

In [ ]:
days = ['1_PO', '2_UT', '3_STR', '4_STVR', '5_PIA', '6_SO']

6. Print WilcoxonResult for: *Uvod*

In [ ]:
for day in days:
  print(stats.wilcoxon(df1[day], df1_estimate[day]))

WilcoxonResult(statistic=54.0, pvalue=0.46911315243545415)
WilcoxonResult(statistic=62.0, pvalue=0.7563688628810696)
WilcoxonResult(statistic=66.0, pvalue=0.9176315934670087)
WilcoxonResult(statistic=52.0, pvalue=0.40804420568711997)
WilcoxonResult(statistic=62.0, pvalue=0.7563688628810696)
WilcoxonResult(statistic=24.0, pvalue=0.022894784183124583)


7. Print WilcoxonResult for: *Studium*

In [ ]:
for day in days:
  print(stats.wilcoxon(df2[day], df2_estimate[day]))

WilcoxonResult(statistic=35.0, pvalue=0.08793560714236243)
WilcoxonResult(statistic=48.0, pvalue=0.3010541816225988)
WilcoxonResult(statistic=55.0, pvalue=0.5014474714870738)
WilcoxonResult(statistic=60.0, pvalue=0.6791156892928161)
WilcoxonResult(statistic=24.0, pvalue=0.022894784183124583)
WilcoxonResult(statistic=24.0, pvalue=0.022894784183124583)


8. Print WilcoxonResult for: *Oznamy*

In [ ]:
for day in days:
  print(stats.wilcoxon(df3[day], df3_estimate[day]))

WilcoxonResult(statistic=46.0, pvalue=0.25529050519961505)
WilcoxonResult(statistic=58.0, pvalue=0.6050949464540121)
WilcoxonResult(statistic=28.0, pvalue=0.038607051842430073)
WilcoxonResult(statistic=55.0, pvalue=0.5014474714870738)
WilcoxonResult(statistic=16.0, pvalue=0.007169734292803208)
WilcoxonResult(statistic=29.0, pvalue=0.04373288761671472)


9. Print WilcoxonResult for: *Fakulta*

In [ ]:
for day in days:
  print(stats.wilcoxon(df4[day], df4_estimate[day]))

WilcoxonResult(statistic=50.0, pvalue=0.35197884167136195)
WilcoxonResult(statistic=45.0, pvalue=0.23432097173768163)
WilcoxonResult(statistic=58.0, pvalue=0.6050949464540121)
WilcoxonResult(statistic=48.0, pvalue=0.3010541816225988)
WilcoxonResult(statistic=36.0, pvalue=0.09798930270791739)
WilcoxonResult(statistic=52.0, pvalue=0.40804420568711997)
